In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Model Preparation

### Data Cleaning

* Get to know your data
* Missing values
    * Options
        * Replace missing values with mean or median `df['column'].fillna(df['column'].mean()`
        * Interpolation
        * Drop rows with missing values
        * Drop columns with missing values
* Outliers
    * Visualize with `plt.boxplot`
    * Tukey's method
    * Options
        * Limit the value of outliers
            * Winsorization
        * Transfrom outliers
            * Log transformation
            * Sqrt tranformation
        * Drop rows with outliers
    

In [ ]:
# Tukey's method to test for outliers
for elem in cols:
    q75, q25 = np.percentile(winsorized_life_data[elem], [75, 25])
    iqr = q75 - q25
    min_val = q25 - (iqr * 1.5)
    max_val = q75 + (iqr * 1.5)
    print(elem)
    print(
        "Outliers:",
        len(
            (
                np.where(
                    (winsorized_life_data[elem] > max_val)
                    | (winsorized_life_data[elem] < min_val)
                )[0]
            )
        ),
        "\n",
    )

### Data Exploration

* Objectives
    * Investigate the properties of variables in our dataset.
    * Discover associations between the variables.
    * Identify useful features that may help us to explain our target variables.
    * Test for assumptions in the data.
* Methods
    * Histograms of each variable
    * `sns.pairplot()`
    * Heat map 
        * `sns.heatmap(df.corr(),vmin=-1,vmax=1,annot=True)`
    * Variance inflation factor to measure multicolinearity

In [ ]:
# Variance inflation factor
def get_vifs(self):
    vifs = []
    for i in range(self.X_const.shape[1]):
        vif = variance_inflation_factor(self.X_const.values, i)
        vifs.append(vif)

    return pd.Series(vifs, index=self.X_const.columns)

In [ ]:
# Encode and scale
def one_hot_encode(X, encode_cols, numeric_cols, index=None):
    other_cols = [c for c in X.columns if c not in encode_cols]

    ct = ColumnTransformer(
        #   Format
        #   [("name of step", what_to_do(), [what columns to do it to])]
        [
            ("one hot encode", OneHotEncoder(drop="first", sparse=False), encode_cols),
            ("scaler", StandardScaler(), numeric_cols),
        ],
        remainder="passthrough",
    )

    ct.fit(X)

    encoded_names = ct.transformers_[0][1].get_feature_names()
    encoded_names = list(encoded_names)

    X_encoded = ct.transform(X)
    X_encoded = pd.DataFrame(X_encoded, columns=encoded_names + other_cols, index=index)

    return X_encoded

### Feature Engineering

* Normality
    * `boxcox(df['column'])`
* Scaling
    * `scaler= StandardScaler()`
    * `scaler.fit_tranform(X)`
    * `from sklearn.preprocessing import StandardScaler`
* SelectKBest
    * `selector = SelectKBest(f_regression, k=5)`
    * `k_best = selector.fit_transform(X, y)`
    * `from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression`
* PCA
    * `pca = PCA()`
    * `pca_X = pca.fit_transform(scaled_X)`
    * `from sklearn.decomposition import PCA`